# Exercise 3: File management

<img src='https://naavre.net/img/hero-light.svg' width=200 align=right>

In this exercise, we learn how to manage files. This includes passing files between cells during the execution of the workflow, and uploading resulting files to MinIO.

_Step 0_

Install dependencies. This exercise uses the `numpy` and `matplotlib` libraries, which need to be installed prior to running the notebook. This is done in the cell below. It doesn’t need to be containerized. 

In [ ]:
!pip install numpy matplotlib

_Step 1_

Create a cell that generates a list of numbers, and saves it to a file in the `/tmp/data/` folder. Saving the file to `/tmp/data/` is crucial, as it is the only way 

For example, we can draw 100000 numbers from a normal distribution:

```python
# ex3 1: Create file
import os
import numpy as np

numbers = np.random.normal(size=100_000)
```

and save it to `/tmp/data/ex3-numbers.out`:

```python
data_file = os.path.join(conf_data_path, "ex3-data.out")
os.makedirs(conf_data_path, exist_ok=True)
np.savetxt(data_file, numbers)
```

_Step 2_

Load the file created by the previous cell, and create a figure from its contents. Save the figure to a file in `/tmp/data/`. 

For example, we can plot probability density with `matplotlib` and save it:

```python
# ex3 2: Plot file
import numpy as np
import matplotlib.pyplot as plt

data = np.loadtxt(data_file)

plt.hist(data, bins=50, density=True, label="Normal distribution")
plt.xlabel("Value [unitless]")
plt.ylabel("Probability density")
plt.title("A plot made with NaaVRE")
plt.legend()

figure_file = os.path.join(conf_data_path, "ex3-figure.pdf")
os.makedirs(conf_data_path, exist_ok=True)
plt.savefig(figure_file)
```

_Step 3_

Upload the file to an external storage service. 

_a._ Follow the [MinIO tutorial](https://docs.google.com/presentation/d/112Vs-vsOonVq1TlC4WprzWR6s9XntOBGgoFGbbT_FDg/edit?usp=sharing) to learn how to login and obtain an access and secret key.

_b._ Add a cell at the **start** of the notebook, creating the following parameters:

```python
param_minio_endpoint = "scruffy.lab.uvalight.net:9000"
param_minio_bucket = "naa-vre-user-data"
param_minio_user_prefix = ""  # Your personal folder, see tutorial
secret_minio_access_key = ""  # Your access key, see tutorial
secret_minio_secret_key = ""  # Your secret key, see tutorial
```

**Warning:** For security reasons, you should not commit notebooks with secrets to Git, or publish them in any way. 

_c._ Add a cell at the **end** of the notebook, which uploads the figure to MinIO:

```python
# ex3 3: Upload file
from minio import Minio
mc = Minio(
    endpoint=param_minio_endpoint,
    access_key=secret_minio_access_key,
    secret_key=secret_minio_secret_key,
    )
mc.fput_object(
    bucket_name=param_minio_bucket,
    file_path=figure_file,
    object_name=f"{param_minio_user_prefix}/ex3-figure.pdf",
    )
```

_Step 4_

Containerize the cells, create a workflow, and run it.